AARYAN JOSHI <br>
BACO19112 <br>
BE - A <br>
LP5 <br>
HPC Mini Project - Mini Project: Implement Huffman Encoding on GPU

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1bekcz_f
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1bekcz_f
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=344f683c4e68ede8d9f0be5350154c4a3798f4ff976229a86d4f8c0ef421dc5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6uec3brh/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [6]:
%load_ext Cython

In [8]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

33 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2023-05-16 21:59:37--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2023-05-16 21:59:38--  https://developer.nvidia.com/downloads/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61

In [9]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [10]:
%%cu
#include <stdio.h>

__global__ void helloWorld() {
    printf("Hello, World from GPU!\n");
}

int main() {
    // Launch CUDA kernel
    helloWorld<<<1, 1>>>();

    // Synchronize GPU threads
    cudaDeviceSynchronize();

    return 0;
}


Hello, World from GPU!



In [11]:
%%cu
#include <stdio.h>

// Node structure for Huffman Tree
struct Node {
    char symbol;
    int frequency;
    Node* left;
    Node* right;
};

// Kernel function for Huffman encoding on GPU
__global__ void huffmanEncoding(char* input, int* output, int inputSize, Node* huffmanTree) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < inputSize) {
        char symbol = input[tid];
        Node* currentNode = huffmanTree;

        // Traverse the Huffman Tree until leaf node is reached
        while (currentNode->left != NULL && currentNode->right != NULL) {
            if (symbol == currentNode->left->symbol) {
                output[tid] = 0;
                currentNode = currentNode->left;
            } else if (symbol == currentNode->right->symbol) {
                output[tid] = 1;
                currentNode = currentNode->right;
            }
        }
    }
}

int main() {
    // Initialize input data
    char input[] = "HELLO CUDA";
    int inputSize = sizeof(input) - 1;

    // Create Huffman Tree
    Node node1 = { 'D', 1, NULL, NULL };
    Node node2 = { 'E', 1, NULL, NULL };
    Node node3 = { 'H', 1, NULL, NULL };
    Node node4 = { 'L', 2, NULL, NULL };
    Node node5 = { 'O', 1, NULL, NULL };
    Node node6 = { 'C', 1, &node1, &node2 };
    Node node7 = { 'U', 1, &node3, &node4 };
    Node node8 = { 'T', 1, &node5, &node6 };
    Node huffmanTree = { '\0', 0, &node7, &node8 };

    // Allocate memory on GPU
    char* d_input;
    int* d_output;
    Node* d_huffmanTree;
    cudaMalloc((void**)&d_input, sizeof(char) * inputSize);
    cudaMalloc((void**)&d_output, sizeof(int) * inputSize);
    cudaMalloc((void**)&d_huffmanTree, sizeof(Node));

    // Copy input data and Huffman Tree to GPU
    cudaMemcpy(d_input, input, sizeof(char) * inputSize, cudaMemcpyHostToDevice);
    cudaMemcpy(d_huffmanTree, &huffmanTree, sizeof(Node), cudaMemcpyHostToDevice);

    // Define CUDA grid and block dimensions
    int blockSize = 256;
    int gridSize = (inputSize + blockSize - 1) / blockSize;

    // Launch CUDA kernel for Huffman encoding
    huffmanEncoding<<<gridSize, blockSize>>>(d_input, d_output, inputSize, d_huffmanTree);

    // Copy the encoded output back to the CPU
    int* encodedOutput = (int*)malloc(sizeof(int) * inputSize);
    cudaMemcpy(encodedOutput, d_output, sizeof(int) * inputSize, cudaMemcpyDeviceToHost);

    // Print the encoded output
    printf("Input: %s\n", input);
    printf("Encoded Output: ");
    for (int i = 0; i < inputSize; i++) {
        printf("%d", encodedOutput[i]);
    }
    printf("\n");

    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_huffmanTree);
    free(encodedOutput);

    return 0;
}


Input: HELLO CUDA
Encoded Output: 00001348955753176850953918673961762595660255996022084

